In [ ]:
# ===== Модули Обновлено =====

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR

import airgen
import numpy as np
import rerun as rr
import time
from math import cos, sin, pi

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR()

In [ ]:
# ===== Подключение к дрону и взлёт =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# ===== Создание клиента сцены =====
scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482445
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482405
BP_PIPCamera_C_2147482393
BP_PIPCamera_C_2147482381
BP_PIPCamera_C_2147482369
BP_PIPCamera_C_2147482357


In [ ]:
# ===== Получение данных LiDAR + визуализация =====
lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Получение и логирование изображения с камеры =====
image_data = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image, _ = image_data  # Получаем изображение и позу
rr.log("drone/camera", rr.Image(rgb_image))

In [ ]:
# ===== Получение и логирование GPS =====

gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))

In [ ]:
from grid.model.perception.detection.rt_detr import RT_DETR
import rerun as rr
import airgen
import numpy as np
import cv2

drone = airgen.MultirotorClient()  # Заменяем AirGenCar на drone

# We will be capturing an image from the AirGen simulator 
# and run model inference on it.

# Получаем изображение с камеры в формате Scene
image_data, _ = drone.getImages("front_center", [airgen.ImageType.Scene])[0]

# image_data теперь содержит изображение. Передаем его в модель.
model = RT_DETR(use_local=True)
result = model.run(input=image_data.copy(), confidence_threshold=0.5)

# Проверяем, есть ли объекты в результате детекции
if result and 'boxes' in result and len(result['boxes']) > 0:
    # Логируем результаты как объекты
    boxes = result['boxes']
    labels = result['labels']
    
    # Рисуем прямоугольники на изображении
    image_with_boxes = image_data.copy()
    
    for i, box in enumerate(boxes):
        x, y, w, h = box
        # Рисуем прямоугольник на изображении
        cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Логируем результат как объект
        rr.log(
            f"drone/obstacle_{i}",
            rr.Boxes2D(np.array([[x, y, w, h]]), labels=["Obstacle"])
        )
    
    # Логируем изображение с обведенными объектами
    rr.log("drone/camera", rr.Image(image_with_boxes))
    
else:
    print("❗ Объекты не обнаружены.")


❗ Объекты не обнаружены.


In [ ]:
# ===== Построение и визуализация кругового маршрута =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

# Стартовая точка
start = client.getMultirotorState().kinematics_estimated.position
takeoff_alt = -15  # минус = вверх в NED
center = start
radius = 10
steps = 12  # количество точек круга

# Построение круга
trajectory = []
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

# Возврат к центру
trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))

# Визуализация маршрута
rr.log("telemetry/path", rr.LineStrips3D([[(p.x_val, p.y_val, p.z_val) for p in trajectory]]))

In [ ]:

# ===== Плавное движение по маршруту и посадка =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))
velocity = 3.0  # Плавная скорость

client.moveOnPathAsync(
    trajectory,
    velocity,
    300,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1, 0
).join()

# Плавная посадка
landing_point = airgen.Vector3r(center.x_val, center.y_val, -2)
client.moveToPositionAsync(landing_point.x_val, landing_point.y_val, landing_point.z_val, 1).join()
client.landAsync().join()


In [ ]:
import airgen
import time

# Подключаемся к дрону
drone = airgen.MultirotorClient()

# Устанавливаем движение вперед
def move_forward(distance=10, velocity=3.0):
    # Получаем текущую позицию
    current_position = drone.getMultirotorState().kinematics_estimated.position

    # Рассчитываем новую позицию
    new_position = airgen.Vector3r(
        current_position.x_val + distance,  # Двигаемся вперед вдоль оси X
        current_position.y_val,             # Остаемся на месте по оси Y
        current_position.z_val              # Остаемся на той же высоте по оси Z
    )

    # Двигаемся к новой позиции с заданной скоростью
    drone.moveToPositionAsync(new_position.x_val, new_position.y_val, new_position.z_val, velocity).join()

# Пример движения на 10 метров вперед с максимальной скоростью 3 м/с
move_forward(distance=10, velocity=3.0)
